# Move to statkit

In [105]:
class General_set_stats(object):
    def __init__(self, from_series=True, seriesStatsClass=None):
        if from_series:
            assert seriesStatsClass, 'If choosen to build from series, must send seriesStatsClass to input'
            self.data = seriesStatsClass.data
            self.get_stats_from_series()

    def get_stats_from_series():
        pass
    
    def base_mean_confidence(self):
        pass

    def count_N_gen(self, n, perc_n):
        pass
    
    def base_mean_interval(self, prob, mean, n, s=None, disp=None, std=None, gen_std=None):
        if not gen_std:
            trust = tinv(prob, n)
            if s:
                std = s #????
            if std:
                std = np.sqrt(n/(n-1)) * std
            if disp:
                std = np.sqrt(n/(n-1)*disp)
        else:
            trust = Laplace_table_inverse(prob)
            std = gen_std
    
        delta = (trust * std) / np.sqrt(n)
        left, right = map(stat_round, (mean-delta, mean+delta))
        return left, right, delta

    def disp_interval(self, prob, n, s=None, disp=None, std=None):
        if not s:
            if std:
                s = np.sqrt(n/(n-1))*std
            elif disp:
                s = np.sqrt(n/(n-1) * disp)
            
        alpha_left, alpha_right, k = (1-prob)/2, (1+prob)/2, n-1
        border = lambda alpha: ((n-1) * s**2) / chiinv(alpha, n)
        borders = (border(a) for a in (alpha_left, alpha_right))
        
        left, right = map(stat_round, map(np.sqrt, borders))
        
        return left, right, alpha_right-alpha_left

    def gen_mean_un_repeat(self, n, disp_2, mean, prob=.9974, N=None, perc_n=None, kind='unrepeat'):
        assert perc_n or N, 'need input data for calculating N general'
        if not N:
            N = n / perc_n
            
        if n < 30:
            trust = tinv(prob, n)
        else:
            trust = Laplace_table_inverse(prob)
        
        if kind == 'unrepeat':
            mu = np.sqrt( (disp_2/n) * (1-n/N) )
        if kind == 'repeat':
            mu = np.sqrt( disp_2/n )
            
        delta = trust * mu
        borders = mean - delta, mean + delta
        left, right = map(stat_round, (borders))
        return left, right, delta

    def rate_disp(self, disp_n, n):
        return n/(n-1) * disp_n

    def share_gen_score(self, n, k, prob=.9974, N=None, perc_n=.05, kind='unrepeat'):
        assert perc_n or N, 'need input data for calculating N general'
        if not N:
            N = n / perc_n
        
        w = k/n
        
        trust = Laplace_table_inverse(prob)
    
        if kind == 'unrepeat':
            mu = np.sqrt( ((w*(1-w))/n) * (1-n/N) )
        if kind == 'repeat':
            mu = np.sqrt( (w*(1-w))/n )
    
        delta = trust * mu
        
        borders = w - delta, w + delta
        left, right = map(stat_round, (borders))
        return left, right, delta

In [101]:
n = 100
k = 65
prob = .9974
perc_n = .01

In [102]:
gen = General_set_stats(from_series=False)

In [103]:
gen.share_gen_score(n, k, perc_n=perc_n)

(0.5071, 0.7929, 0.14291939375068205)

# Working

In [3]:
from statkit import (seriesBuilder as SB, stattools)
from statkit.functions import *

import numpy as np

In [87]:
data = np.array([
    (96, 98, 5),
    (98, 100, 30),
    (100, 102, 60),
    (102, 104, 5)
])

In [88]:
series = SB.intervalVariationSeries(data)

In [89]:
series.table

,left,right,x,n,h,w,n/h,w/h,w_cum
0,96.0,98.0,97.0,5.0,2.0,0.05,2.5,0.025,0.05
1,98.0,100.0,99.0,30.0,2.0,0.30,15.0,0.150,0.35
2,100.0,102.0,101.0,60.0,2.0,0.60,30.0,0.300,0.95
3,102.0,104.0,103.0,5.0,2.0,0.05,2.5,0.025,1.00
Total,396.0,404.0,400.0,100.0,8.0,1.00,50.0,0.500,2.35


In [90]:
stats = stattools.Stats(series)

In [91]:
stats.show_stats()

,mean,mode,median,range,abs_dev,disp,UB disp,std,UB_std,coeff var
0,100.3,100.67,100.5,6.0,1.11,1.71,1.73,1.31,1.31,1.3


# Other

In [6]:
Laplace_table_inverse(.9974)

3.0115